## Introduction
Throughout this notebook I use the paper _"From reads to insight: a hitchhiker’s guide to ATAC-seq data analysis"_ as a guide. It was published in February this year and gives a detailed description of the ATAC-seq workflow, pointing out potential pitfalls and describing the different tools which can be used at each stage in the pipeline. Here is a [link](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-020-1929-3) to the paper. 

Eventually, I decided to stick with the pipeline given in the reference notebook, but I considered other tools at each stage. 

## Getting set up

In [1]:
mkdir -p /mnt/storage/$USER/jupyternotebooks/epigenomics/ATAC/
cd /mnt/storage/$USER/jupyternotebooks/epigenomics/ATAC/

In [2]:
vdb-config -s /repository/user/cache-disabled=true

In [ ]:
fastq-dump --split-files SRR9113345 SRR9113348

2020-12-08T15:20:38 fastq-dump.2.9.6 sys: timeout exhausted while reading file within network system module - mbedtls_ssl_read returned -76 ( NET - Reading information from the socket failed )
2020-12-08T15:21:47 fastq-dump.2.9.6 sys: timeout exhausted while reading file within network system module - mbedtls_ssl_read returned -76 ( NET - Reading information from the socket failed )
2020-12-08T15:25:20 fastq-dump.2.9.6 sys: timeout exhausted while reading file within network system module - mbedtls_ssl_read returned -76 ( NET - Reading information from the socket failed )


# Quality Control
##  Pre-alignment quality control using FastQC
I run FastQC to check the quality of the reads. In the previous assignment I discussed each of the tests in detail so I won't do that here. Due to the ubiquitous use of Illumina’s Nextera library for ATAC-seq, overrepresentation of Nextera sequencing adapters is often observed and should be removed for accurate read alignment. The FastQC reports showed that each run failed the `per base sequence content` and `kmer content` tests. The QC report indicated that the quality of the reads were fine and the reads didn't need to be trimmed.

In [5]:
/usr/bin/fastqc -o . *.fastq

Started analysis of SRR9113339_1.fastq
Approx 5% complete for SRR9113339_1.fastq
Approx 10% complete for SRR9113339_1.fastq
Approx 15% complete for SRR9113339_1.fastq
Approx 20% complete for SRR9113339_1.fastq
Approx 25% complete for SRR9113339_1.fastq
Approx 30% complete for SRR9113339_1.fastq
Approx 35% complete for SRR9113339_1.fastq
Approx 40% complete for SRR9113339_1.fastq
Approx 45% complete for SRR9113339_1.fastq
Approx 50% complete for SRR9113339_1.fastq
Approx 55% complete for SRR9113339_1.fastq
Approx 60% complete for SRR9113339_1.fastq
Approx 65% complete for SRR9113339_1.fastq
Approx 70% complete for SRR9113339_1.fastq
Approx 75% complete for SRR9113339_1.fastq
Approx 80% complete for SRR9113339_1.fastq
Approx 85% complete for SRR9113339_1.fastq
Approx 90% complete for SRR9113339_1.fastq
Approx 95% complete for SRR9113339_1.fastq
Analysis complete for SRR9113339_1.fastq
Started analysis of SRR9113339_2.fastq
Approx 5% complete for SRR9113339_2.fastq
Approx 10% complete for

# Alignment
Alignment was done using Bowtie2.

In [6]:
bowtie2 -x /mnt/storage/data/resources/bowtie2/hg19 -1 SRR9113339_1.fastq -2 SRR9113339_2.fastq -S ATAC_DMSO.sam
bowtie2 -x /mnt/storage/data/resources/bowtie2/hg19 -1 SRR9113342_1.fastq -2 SRR9113342_2.fastq -S ATAC_O.sam
bowtie2 -x /mnt/storage/data/resources/bowtie2/hg19 -1 SRR9113345_1.fastq -2 SRR9113345_2.fastq -S ATAC_OT.sam
bowtie2 -x /mnt/storage/data/resources/bowtie2/hg19 -1 SRR9113348_1.fastq -2 SRR9113348_2.fastq -S ATAC_rebound.sam


34676467 reads; of these:
  34676467 (100.00%) were paired; of these:
    2715124 (7.83%) aligned concordantly 0 times
    19068002 (54.99%) aligned concordantly exactly 1 time
    12893341 (37.18%) aligned concordantly >1 times
    ----
    2715124 pairs aligned concordantly 0 times; of these:
      798052 (29.39%) aligned discordantly 1 time
    ----
    1917072 pairs aligned 0 times concordantly or discordantly; of these:
      3834144 mates make up the pairs; of these:
        2362395 (61.61%) aligned 0 times
        596813 (15.57%) aligned exactly 1 time
        874936 (22.82%) aligned >1 times
96.59% overall alignment rate
29428293 reads; of these:
  29428293 (100.00%) were paired; of these:
    2881950 (9.79%) aligned concordantly 0 times
    18949636 (64.39%) aligned concordantly exactly 1 time
    7596707 (25.81%) aligned concordantly >1 times
    ----
    2881950 pairs aligned concordantly 0 times; of these:
      756404 (26.25%) aligned discordantly 1 time
    ----
    21255

Convert to Bam file and create an index:

In [9]:
samtools sort -o ATAC_DMSO.bam ATAC_DMSO.sam
samtools sort -o ATAC_O.bam ATAC_O.sam
samtools sort -o ATAC_OT.bam ATAC_OT.sam
samtools sort -o ATAC_rebound.bam ATAC_rebound.sam

[bam_sort_core] merging from 16 files and 1 in-memory blocks...
[bam_sort_core] merging from 13 files and 1 in-memory blocks...
[bam_sort_core] merging from 12 files and 1 in-memory blocks...
[bam_sort_core] merging from 13 files and 1 in-memory blocks...


In [10]:
samtools index ATAC_DMSO.bam
samtools index ATAC_O.bam
samtools index ATAC_OT.bam
samtools index ATAC_rebound.bam

# Post-alignment processing and quality control
After sequence alignment, as in most DNA sequencing data, basic metrics of the aligned BAM file, such as unique mapping reads/rates, duplicated read percentages, and fragment size distribution can be collected using Picard and SAMtools. Reads should be removed if they are improperly paired or of low mapping quality. Additionally, the mitochondrial genome, which is more accessible due to the lack of chromatin packaging, and the ENCODE blacklisted regions often have extremely high read coverage, and should also be discarded. 

The overall alignment rates for the four conditions were 97%, 95%, 92% and 95%. First I used `samtools` to look at basic statistics, which looked normal.

In [11]:
samtools idxstats ATAC_DMSO.bam

chr1	249250621	5774239	30283
chr2	243199373	2984561	20596
chr3	198022430	2274993	12130
chr4	191154276	1414891	8768
chr5	180915260	2556082	15641
chr6	171115067	1762962	9624
chr7	159138663	2567996	14552
chr8	146364022	1317030	8183
chr9	141213431	1522265	9110
chr10	135534747	1448691	10194
chr11	135006516	2186417	13956
chr12	133851895	1634793	9263
chr13	115169878	673477	3968
chr14	107349540	1002588	5754
chr15	102531392	1031752	5848
chr16	90354753	1494110	8240
chr17	81195210	1768429	11851
chr18	78077248	470133	3087
chr19	59128983	1448025	9156
chr20	63025520	1005243	5453
chr21	48129895	316525	2252
chr22	51304566	772098	4242
chrX	155270560	815024	6173
chrY	59373566	43122	1511
chrM	16571	28511826	163549
chr1_gl000191_random	106433	1809	13
chr1_gl000192_random	547496	5560	25
chr4_gl000193_random	189789	1631	25
chr4_gl000194_random	191469	1588	10
chr7_gl000195_random	182896	11400	50
chr8_gl000196_random	38914	103	1
chr8_gl000197_random	37175	154	0
chr9_gl000198_random	90085	502	13
chr9_gl000199_

In [17]:
samtools flagstat ATAC_DMSO.bam
samtools flagstat ATAC_O.bam
samtools flagstat ATAC_OT.bam
samtools flagstat ATAC_rebound.bam

69352934 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
66990539 + 0 mapped (96.59% : N/A)
69352934 + 0 paired in sequencing
34676467 + 0 read1
34676467 + 0 read2
63922686 + 0 properly paired (92.17% : N/A)
66593878 + 0 with itself and mate mapped
396661 + 0 singletons (0.57% : N/A)
642388 + 0 with mate mapped to a different chr
275037 + 0 with mate mapped to a different chr (mapQ>=5)
58856586 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
55765818 + 0 mapped (94.75% : N/A)
58856586 + 0 paired in sequencing
29428293 + 0 read1
29428293 + 0 read2
53092686 + 0 properly paired (90.21% : N/A)
55446294 + 0 with itself and mate mapped
319524 + 0 singletons (0.54% : N/A)
475648 + 0 with mate mapped to a different chr
205692 + 0 with mate mapped to a different chr (mapQ>=5)
54339592 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
500

There are additional ATAC-seq-specific quality metrics that should also be evaluated. Typically, a successful ATAC-seq experiment should generate a fragment size distribution plot with decreasing and periodical peaks corresponding to the nucleosome-free regions (NFR). Fragments from the NFR are expected to be enriched around the transcription start site (TSS) of genes, while fragments from nucleosome-bound regions are expected to be depleted at TSS with a slight enrichment of flanking regions around TSS. These can be evaluated with the R package ATACseqQC. ATACseqQC can also be used to filter the mitochondrial genome and ENCODE blacklisted regions. Lastly, reads should be shifted + 4 bp and − 5 bp for positive and negative strand respectively, to account for the 9-bp duplication created by DNA repair of the nick by Tn5 transposase and achieve base-pair resolution of TF footprint and motif-related analyses.

The ATACseqQC package is not installed on the server so I tried to run this analysis locally but it was too computationally intensive. Therefore I don't have any results to present but I wanted to show that I had looked into this and learned from it at least.

DeepTools can also be used to perform some basic quality control, as well as look at PCA plots and heatmaps:

In [27]:
multiBamSummary bins --bamfiles ATAC_DMSO.bam ATAC_O.bam ATAC_OT.bam ATAC_rebound.bam -o readCounts

Number of bins found: 310124


In [29]:
plotPCA -in readCounts \
-o PCA_readCounts.png \
-T "PCA of read counts"

The PCA plot shows that the DMSO, Osimertinib, and rebound are all similar and that the chromatin in cells treated with osimertinib and trametinib is different. 
![PCA](images/PCA_readCounts.png)

In [39]:
plotCorrelation --corData readCounts \
                --corMethod pearson  \
                --whatToPlot heatmap \
                --removeOutliers     \
                -o heatmap-pearson.png
                
plotCorrelation --corData readCounts \
                --corMethod spearman \
                --whatToPlot heatmap \
                -o heatmap-spearman.png

total/filtered/left: 310124/7/310117

Outliers were detected in the data. They will be removed to avoid bias in the pearson correlation.


A heatmap was plotted using Pearson correlation shich shows again the similarity between different conditions. Again we see that OT is most different from the other three conditions which are similar. The correlation plot shows that REBOUND and DMSO are most similar to each other while OT is the least similar to all other three conditions. This fits with the findings in the paper that OT treated cells have a distinct chromatin state which reverses to its original upon state upon washout.

![heatmap-pearson](images/heatmap-pearson.png)

In [41]:
plotCorrelation --corData readCounts \
                --corMethod pearson  \
                --whatToPlot scatterplot \
                --removeOutliers     \
                -o scatter-pearson.png
                
plotCorrelation --corData readCounts \
                --corMethod spearman \
                --whatToPlot scatterplot \
                -o scatter-spearman.png

total/filtered/left: 310124/7/310117

Outliers were detected in the data. They will be removed to avoid bias in the pearson correlation.


The scatter plots tell a similar story.
![scatter-pearson](images/scatter-pearson.png)

# Core Analysis: Peak Calling
I perform peak calling using MACS2.

In [16]:
macs2 predictd -i ATAC_DMSO.bam
macs2 predictd -i ATAC_O.bam
macs2 predictd -i ATAC_OT.bam
macs2 predictd -i ATAC_rebound.bam

INFO  @ Thu, 10 Dec 2020 11:57:49: # read alignment files... 
INFO  @ Thu, 10 Dec 2020 11:57:49: # read treatment tags... 
INFO  @ Thu, 10 Dec 2020 11:57:49: Detected format is: BAM 
INFO  @ Thu, 10 Dec 2020 11:57:49: * Input file is gzipped. 
INFO  @ Thu, 10 Dec 2020 11:57:51:  1000000 
INFO  @ Thu, 10 Dec 2020 11:57:53:  2000000 
INFO  @ Thu, 10 Dec 2020 11:57:54:  3000000 
INFO  @ Thu, 10 Dec 2020 11:57:56:  4000000 
INFO  @ Thu, 10 Dec 2020 11:57:58:  5000000 
INFO  @ Thu, 10 Dec 2020 11:58:00:  6000000 
INFO  @ Thu, 10 Dec 2020 11:58:02:  7000000 
INFO  @ Thu, 10 Dec 2020 11:58:03:  8000000 
INFO  @ Thu, 10 Dec 2020 11:58:05:  9000000 
INFO  @ Thu, 10 Dec 2020 11:58:07:  10000000 
INFO  @ Thu, 10 Dec 2020 11:58:09:  11000000 
INFO  @ Thu, 10 Dec 2020 11:58:10:  12000000 
INFO  @ Thu, 10 Dec 2020 11:58:12:  13000000 
INFO  @ Thu, 10 Dec 2020 11:58:14:  14000000 
INFO  @ Thu, 10 Dec 2020 11:58:16:  15000000 
INFO  @ Thu, 10 Dec 2020 11:58:18:  16000000 
INFO  @ Thu, 10 Dec 2020 11:5

INFO  @ Thu, 10 Dec 2020 12:01:40: * Input file is gzipped. 
INFO  @ Thu, 10 Dec 2020 12:01:42:  1000000 
INFO  @ Thu, 10 Dec 2020 12:01:44:  2000000 
INFO  @ Thu, 10 Dec 2020 12:01:45:  3000000 
INFO  @ Thu, 10 Dec 2020 12:01:47:  4000000 
INFO  @ Thu, 10 Dec 2020 12:01:49:  5000000 
INFO  @ Thu, 10 Dec 2020 12:01:51:  6000000 
INFO  @ Thu, 10 Dec 2020 12:01:53:  7000000 
INFO  @ Thu, 10 Dec 2020 12:01:54:  8000000 
INFO  @ Thu, 10 Dec 2020 12:01:56:  9000000 
INFO  @ Thu, 10 Dec 2020 12:01:58:  10000000 
INFO  @ Thu, 10 Dec 2020 12:02:00:  11000000 
INFO  @ Thu, 10 Dec 2020 12:02:02:  12000000 
INFO  @ Thu, 10 Dec 2020 12:02:03:  13000000 
INFO  @ Thu, 10 Dec 2020 12:02:05:  14000000 
INFO  @ Thu, 10 Dec 2020 12:02:07:  15000000 
INFO  @ Thu, 10 Dec 2020 12:02:09:  16000000 
INFO  @ Thu, 10 Dec 2020 12:02:10:  17000000 
INFO  @ Thu, 10 Dec 2020 12:02:12:  18000000 
INFO  @ Thu, 10 Dec 2020 12:02:14:  19000000 
INFO  @ Thu, 10 Dec 2020 12:02:16:  20000000 
INFO  @ Thu, 10 Dec 2020 12:

I did some research into the correct macs2 options for peak calling ATAC-seq data. A number of forums on BioStars gave different recommendations. The debate revolves around using the `--nomodel --shift --extsize` options or the `--format BAMPE` option. Eventually, I found the answer on the official Harvard guideline for ATAC-seq data analysis:

An important consideration when using MACS2 is deciding which types of alignments should be analyzed and how those alignments should be interpreted. The analysis mode is set by the -f argument. There are two primary options:

- Analyze only properly paired alignments, but ignore R2 reads and treat R1 reads as singletons. MACS2 creates a model of the fragment lengths and extends the 3' ends of the R1 reads to the calculated average length. An alternative is to skip this model building and instead extend each read to a specified length (e.g., `--nomodel --extsize 300` for 300bp fragments). The value of the length parameter is usually determined from the average size during library preparation. However, neither of these approaches utilizes the value of paired-end sequencing, which defines both fragment ends.

- Analyze only properly paired alignments with `-f BAMPE`. Here, the fragments are defined by the paired alignments' ends, and there is no modeling or artificial extension. Singleton alignments are ignored. This is the preferred option for using only properly paired alignments.

A MACS2 developer also noted: _"If you followed original protocol for ATAC-Seq, you should get Paired-End reads. If so, I would suggest you just use `--format BAMPE` to let MACS2 pileup the whole fragments in general. But if you want to focus on looking for where the 'cutting sites' are, then `--nomodel --shift -100 --extsize 200` should work."_

Look at the `flagstat` outputs, we have over 90% properly paired reads, so I decided to use the `--format BAMPE` option.

The [official Harvard guidelines](https://informatics.fas.harvard.edu/atac-seq-guidelines.html#peak) as of this year actually recommends using `Genrich` instead of `MACS2`. This is because Genrich was designed to be able to run all of the post-alignment steps through peak-calling with one command (removal of mitochondiral reads, removal of PCr duplicates, etc.). Most importantly, Genrich provides an ATAC-seq analysis mode in which, rather than inferring the full fragments from the alignments, intervals are interpreted that are centered on transposase cut sites. MACS2 was originally designed for ChIP-seq analysis and so is not perfectly suited to ATAC-seq.

In [1]:
macs2 callpeak -B -t ATAC_DMSO.bam -n DMSO_peaks --format BAMPE
macs2 callpeak -B -t ATAC_O.bam -n O_peaks --format BAMPE
macs2 callpeak -B -t ATAC_OT.bam -n OT_peaks --format BAMPE
macs2 callpeak -B -t ATAC_rebound.bam -n rebound_peaks --format BAMPE

INFO  @ Fri, 11 Dec 2020 00:18:23: 
# Command line: callpeak -B -t ATAC_O.bam -n O_peaks --format BAMPE
# ARGUMENTS LIST:
# name = O_peaks
# format = BAMPE
# ChIP-seq file = ['ATAC_O.bam']
# control file = None
# effective genome size = 2.70e+09
# band width = 300
# model fold = [5, 50]
# qvalue cutoff = 5.00e-02
# The maximum gap between significant sites is assigned as the read length/tag size.
# The minimum length of peaks is assigned as the predicted fragment length "d".
# Larger dataset will be scaled towards smaller dataset.
# Range for calculating regional lambda is: 10000 bps
# Broad region calling is off
# Paired-End mode is on
 
INFO  @ Fri, 11 Dec 2020 00:18:23: #1 read fragment files... 
INFO  @ Fri, 11 Dec 2020 00:18:23: #1 read treatment fragments... 
INFO  @ Fri, 11 Dec 2020 00:18:27:  1000000 
INFO  @ Fri, 11 Dec 2020 00:18:31:  2000000 
INFO  @ Fri, 11 Dec 2020 00:18:35:  3000000 
INFO  @ Fri, 11 Dec 2020 00:18:40:  4000000 
INFO  @ Fri, 11 Dec 2020 00:18:44:  5000000 

INFO  @ Fri, 11 Dec 2020 07:36:55:  3000000 
INFO  @ Fri, 11 Dec 2020 07:36:59:  4000000 
INFO  @ Fri, 11 Dec 2020 07:37:04:  5000000 
INFO  @ Fri, 11 Dec 2020 07:37:08:  6000000 
INFO  @ Fri, 11 Dec 2020 07:37:12:  7000000 
INFO  @ Fri, 11 Dec 2020 07:37:16:  8000000 
INFO  @ Fri, 11 Dec 2020 07:37:20:  9000000 
INFO  @ Fri, 11 Dec 2020 07:37:25:  10000000 
INFO  @ Fri, 11 Dec 2020 07:37:29:  11000000 
INFO  @ Fri, 11 Dec 2020 07:37:33:  12000000 
INFO  @ Fri, 11 Dec 2020 07:37:37:  13000000 
INFO  @ Fri, 11 Dec 2020 07:37:42:  14000000 
INFO  @ Fri, 11 Dec 2020 07:37:46:  15000000 
INFO  @ Fri, 11 Dec 2020 07:37:50:  16000000 
INFO  @ Fri, 11 Dec 2020 07:37:54:  17000000 
INFO  @ Fri, 11 Dec 2020 07:37:58:  18000000 
INFO  @ Fri, 11 Dec 2020 07:38:02:  19000000 
INFO  @ Fri, 11 Dec 2020 07:38:05:  20000000 
INFO  @ Fri, 11 Dec 2020 07:38:09:  21000000 
INFO  @ Fri, 11 Dec 2020 07:38:13:  22000000 
INFO  @ Fri, 11 Dec 2020 07:38:16:  23000000 
INFO  @ Fri, 11 Dec 2020 07:38:20:  24000

# Advanced analysis
Because by its nature ATAC-seq reveals multiple aspects of transcriptional regulation, the next step of the analysis involves interpretation at four different levels: peak, motif, nucleosome, and TF footprint.

## Peaks
### Peak differential analysis
First I get the read depth for each condition:

In [15]:
egrep "fragments after filtering in treatment|fragment size is determined as" DMSO_peaks_peaks.xls
egrep "fragments after filtering in treatment|fragment size is determined as" O_peaks_peaks.xls
egrep "fragments after filtering in treatment|fragment size is determined as" OT_peaks_peaks.xls
egrep "fragments after filtering in treatment|fragment size is determined as" rebound_peaks_peaks.xls

# fragment size is determined as 160 bps
# fragments after filtering in treatment: 17309886
# fragment size is determined as 163 bps
# fragments after filtering in treatment: 19126680
# fragment size is determined as 184 bps
# fragments after filtering in treatment: 21693181
# fragment size is determined as 163 bps
# fragments after filtering in treatment: 16300748


Then I run `bdgdiff` for differential peak detection:

In [14]:
# osimertinib vs control
macs2 bdgdiff \
    --t1 O_peaks_treat_pileup.bdg \
    --c1 O_peaks_control_lambda.bdg \
    --t2 DMSO_peaks_treat_pileup.bdg \
    --c2 DMSO_peaks_control_lambda.bdg \
    --d1 19126680 \
    --d2 17309886 \
    -g 60 \
    -l 160 \
    --o-prefix diff_O_vs_DMSO

INFO  @ Wed, 16 Dec 2020 15:51:37: Read and build treatment 1 bedGraph... 
INFO  @ Wed, 16 Dec 2020 15:51:57: Read and build control 1 bedGraph... 
INFO  @ Wed, 16 Dec 2020 15:52:20: Read and build treatment 2 bedGraph... 
INFO  @ Wed, 16 Dec 2020 15:52:38: Read and build control 2 bedGraph... 
INFO  @ Wed, 16 Dec 2020 15:55:07: Write peaks... 
INFO  @ Wed, 16 Dec 2020 15:55:07: Done 


In [16]:
# osimertinib + trametinib vs osimertinib
macs2 bdgdiff \
    --t1 OT_peaks_treat_pileup.bdg \
    --c1 OT_peaks_control_lambda.bdg \
    --t2 O_peaks_treat_pileup.bdg \
    --c2 O_peaks_control_lambda.bdg \
    --d1 21693181 \
    --d2 19126680 \
    -g 60 \
    -l 170 \
    --o-prefix diff_OT_vs_O

INFO  @ Wed, 16 Dec 2020 15:58:21: Read and build treatment 1 bedGraph... 
INFO  @ Wed, 16 Dec 2020 15:58:44: Read and build control 1 bedGraph... 
INFO  @ Wed, 16 Dec 2020 15:59:09: Read and build treatment 2 bedGraph... 
INFO  @ Wed, 16 Dec 2020 15:59:28: Read and build control 2 bedGraph... 
INFO  @ Wed, 16 Dec 2020 16:02:06: Write peaks... 
INFO  @ Wed, 16 Dec 2020 16:02:06: Done 


In [26]:
# osimertinib + trametinib vs control
macs2 bdgdiff \
    --t1 OT_peaks_treat_pileup.bdg \
    --c1 OT_peaks_control_lambda.bdg \
    --t2 DMSO_peaks_treat_pileup.bdg \
    --c2 DMSO_peaks_control_lambda.bdg \
    --d1 21693181 \
    --d2 17309886 \
    -g 60 \
    -l 170 \
    --o-prefix diff_OT_vs_DMSO

INFO  @ Wed, 16 Dec 2020 18:01:57: Read and build treatment 1 bedGraph... 
INFO  @ Wed, 16 Dec 2020 18:02:20: Read and build control 1 bedGraph... 
INFO  @ Wed, 16 Dec 2020 18:02:46: Read and build treatment 2 bedGraph... 
INFO  @ Wed, 16 Dec 2020 18:03:04: Read and build control 2 bedGraph... 
INFO  @ Wed, 16 Dec 2020 18:05:46: Write peaks... 
INFO  @ Wed, 16 Dec 2020 18:05:46: Done 


In [19]:
# rebound vs osimertinib + trametinib
macs2 bdgdiff \
    --t1 rebound_peaks_treat_pileup.bdg \
    --c1 rebound_peaks_control_lambda.bdg \
    --t2 OT_peaks_treat_pileup.bdg \
    --c2 OT_peaks_control_lambda.bdg \
    --d1 16300748 \
    --d2 21693181 \
    -g 60 \
    -l 170 \
    --o-prefix diff_rebound_vs_OT

INFO  @ Wed, 16 Dec 2020 17:26:16: Read and build treatment 1 bedGraph... 
INFO  @ Wed, 16 Dec 2020 17:26:33: Read and build control 1 bedGraph... 
INFO  @ Wed, 16 Dec 2020 17:26:54: Read and build treatment 2 bedGraph... 
INFO  @ Wed, 16 Dec 2020 17:27:16: Read and build control 2 bedGraph... 
INFO  @ Wed, 16 Dec 2020 17:29:52: Write peaks... 
INFO  @ Wed, 16 Dec 2020 17:29:52: Done 


## Motifs
In the original publication, the authors used HOMER for MOTIF analysis. Here I use i-CisTarget instead. The authors found that a motif analysis of OT-treated dormant cells versus control cells revealed that the three most significantly enriched motifs were the consensus sites for TEAD family transcription factors, suggesting that the OT-induced epigenetic state is associated with increased TEAD transcription factor binding. I tried to verify this using i-cisTarget to compare the epigenetic states of OT-treated cells vs DMSO cells. The setup and the results of the analysis are given below:

![icis-settings](images/OT_vs_DMSO_settings.png)
![icis-results](images/OT_vs_DMSO_results.png)

The analysis did not find consensus sites for TEAD transcription factors to be significantly enriched. PHF8 was given as the transcription factor associated with the motif with the highest normalised enrichment score. PHF8 is a histone demethylase which induces an EMT-like process by upregulating key EMT transcription factors SNAIL and ZEB1. Both SNA1L and ZEB1 were discussed in the introduction:

_"YAP has been reported to mediate EMT and to directly bind to canonical EMT transcription factors, including SNAIL, SLUG, and ZEB1. The TEAD transcription factors serve as canonical partners for the Hippo pathway effector YAP, which has been associated with resistance to EGFR TKIs in EGFR-mutant NSCLC."_

The authors of the original paper do not reference or discuss PHF8. The motif enrichment results from i-cisTarget indicate that the epigenetic changes acquired in dormancy also lead to the upregulation of PHF8, which in turn upregulates the key EMT transcription factors that the authors were discussing. This suggests that the OT-induced epigenetic state is associated not only with increased TEAD transcription factor binding, but also with upregulation of PHF8. I could not find anything in the literature relating PHF8 to EMT in dormancy, and so perhaps this could be an interesting research question.

As i-CisTarget did not find that consensus sites for TEAD family transcription factors were enriched for OT_vs_DMSO, I instead tried to run a comparative analysis comparing the i-cisTarget analyses for both OT_vs_DMSO and O_vs_DMSO, hoping to find something insightful by comparing the enriched motifs in either analysis (see below). Unfortunately I could not find anything interesting and the enriched motifs were very similar across both conditions, with no obvious motifs significantly more enriched in one analysis.

![comparative-analysis](images/comparative-analysis.png)

Naturally, I began to question the steps so far, wondering if there may be some error in my analysis. I ran each of the previous steps again and ended up with the same results. To test if TEAD family transcription factors were significantly enriched in OT cells I searched for putative binding sites for TEAD family transcription factors and then used DeepTools to view the heatmap across different conditions. To do this I:

1. Searched the JASPAR database for TEAD family transcription factor motifs. These were then downloaded in MEME format (see JASPAR results below).
2. Ran the MEME output through FIMO to find putative binding sites in GFF3 format (tutorial given [here](https://www.researchgate.net/publication/49844689_FIMO_Scanning_for_occurrences_of_a_given_motif)).
3. Convert the GFF3 file to BED format with `gff2bed` from the BEDOPS kit so it can be used with DeepTools.
4. Use this BED file with DeepTools `plotheatmap` to see if there is a difference in accessibility at these binding sites.

![JASPAR](images/JASPAR.png)
![FIMO](images/FIMO.png)

I also used MotifMap to search for TEAD transcription factor binding motifs. MotifMap gives the putative binding sites for the transcription factor and allows the genomic locations to be easily exported as a BED file or to UCSC genome browser. Unfortunately they only had TEAD1, and didn't have TEAD2, TEAD3 or TEAD4.

![MotifMap](images/motif-map.png)

The DeepTools analysis using the TEAD1 map from MotifMap is given below.

In [42]:
computeMatrix scale-regions \
    -S ATAC_DMSO.bw ATAC_OT.bw \
    -R TEAD-motif.bed \
    -a 2000 \
    -b 2000 \
    -out TEAD-peaks-ATAC.tab.gz

In [43]:
plotHeatmap \
    -m TEAD-peaks-ATAC.tab.gz \
    -out TEAD-peaks-ATAC.png \
    --heatmapHeight 15 \
    --refPointLabel peak.center \
    --regionsLabel peaks \
    --plotTitle 'ATAC-seq TEAD motif signal'

![TEAD peaks](images/TEAD-peaks-ATAC.png)

The heatmap shows a small but distinct increase in accessibility at regions which contain the consensus sequence for TEAD1, which fits with the results given in the paper. It is possible that the results I am getting differ from the paper because I am only using one biological replicate for each condition, whereas they have 3 replicates in their analysis. Perhaps re-running this analysis with the replicates would lead to results which are more similar to those in the paper.

The DeepTools analysis using all TEAD motifs generated from JASPAR and FIMO is given below. There were problems with using BEDOPS to make the bed file from GFF3 file, so I instead tried to convert it into a BED file from the FIMO tsv file.

In [144]:
head -n 10 fimo.tsv

motif_id	motif_alt_id	sequence_name	start	stop	strand	score	p-value	q-value	matched_sequence
MA1121.1	TEAD2	chr2	177482	177494	+	17.7685	6.92e-09	0.0793	ccacattccagcc
MA1121.1	TEAD2	chr1_gl000192_random	246266	246278	+	17.7685	6.92e-09	0.0793	ccacattccagcc
MA1121.1	TEAD2	chr17_ctg5_hap1	495988	496000	+	17.7685	6.92e-09	0.0793	ccacattccagcc
MA1121.1	TEAD2	chr16	533498	533510	+	17.7685	6.92e-09	0.0793	ccacattccagcc
MA1121.1	TEAD2	chr12	591999	592011	+	17.7685	6.92e-09	0.0793	ccacattccagcc
MA1121.1	TEAD2	chr12	681204	681216	+	17.7685	6.92e-09	0.0793	CCACATTCCAGCC
MA1121.1	TEAD2	chr20	684752	684764	+	17.7685	6.92e-09	0.0793	ccacattccagcc
MA1121.1	TEAD2	chr6	816670	816682	+	17.7685	6.92e-09	0.0793	ccacattccagcc
MA1121.1	TEAD2	chr20	1456054	1456066	+	17.7685	6.92e-09	0.0793	ccacattccagcc


In [145]:
# gff2bed < fimo.gff > TEAD_ALL.bed
#cut -f1-3 TEAD_ALL.bed > TEAD_all.bed

# remove chromosomes which are not standard. E.g. chr6_apd_hap1
#awk '$1~/^.{4,5}\s' TEAD_all.bed > TEAD_all_filtered.bed

View processed BED file:

In [151]:
head -n 10 TEAD_all.bed

chr1	36405	36418
chr1	36407	36417
chr1	67507	67519
chr1	103657	103670
chr1	107865	107877
chr1	131166	131179
chr1	131168	131178
chr1	139997	140010
chr1	231033	231046
chr1	249974	249987


DeepTools analysis using the processed BED file:

In [ ]:
computeMatrix scale-regions \
    -S ATAC_DMSO.bw ATAC_OT.bw \
    -R TEAD_all.bed \
    -a 2000 \
    -b 2000 \
    -out TEAD-all-peaks-ATAC.tab.gz

In [ ]:
plotHeatmap \
    -m TEAD-all-peaks-ATAC.tab.gz \
    -out TEAD-all-peaks-ATAC.png \
    --heatmapHeight 15 \
    --refPointLabel peak.center \
    --regionsLabel peaks \
    --plotTitle 'ATAC-seq TEAD motif signal'

I used DeepTools' `plotEnrichment` function to calculate the signal enrichment for regions with the TEAD1 motif. 

In [154]:
plotEnrichment -b ATAC_DMSO.bam ATAC_O.bam ATAC_OT.bam ATAC_rebound.bam \
--BED TEAD-motif.bed \
--labels DMSO O OT REBOUND \
-o enrichment.png

# DISCUSSION
The analyses performed above agree with the results presented in the paper in some respects, and don't in others. The DeepTools PCA plots and correlation heatmaps indicate that the OT-treated cells enter a distinct epigenetic state in dormancy. The author's of this paper presented a motif analysis which found that the TEAD-family transcription factor binding sites were more accessible. The TEAD-family transcription factors are known to upregulate canonical EMT transcription factors SNA1L, SLUG and ZEB1, which leads to an increased metastatic potential in dormant NSLSC cells.

The motif analysis performed here using i-cisTarget did not find the TEAD-family transcription factors to be the most significantly upregulated, and instead found that consensus binding sites for PHF8 were significantly upregulated. PHF8 is also known to upregulate SNA1L and SLUG, leading to increased metastatic potential. This is interesting and might be something worth looking into. To confirm that consensus binding sites for TEAD-family transcription factors were upregulated, DeepTools plotHeatMap function was used to determine whether chromatin accessibility changes at these binding sites between DMSO cells and OT-treated cells. This confirmed that TEAD was in fact upregulated in these regions.

Unfortunately I did not have the time to also run the ChIP-seq analysis that was also a key part of this paper. However I learned much about the ATAC-seq pipeline and tried my best to be as thorough as possible, so I'm happy nonetheless.

Merry Christmas!

![Christmas](images/chrysler.png)